# Import Required Modules

In [ ]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)

In [ ]:
# Generate plots inline
%matplotlib inline

import json
import os

# Support for plotting
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import trappy

# Target Configuration

The first step in the LISA's analysis workflow is to build the so called *Test Environment*, an object that given a set of information on the target device takes care of:

- connecting to the target device (via adb in case of Android device, via SSH otherwise)
- setting the directory where data shall be stored
- deploying the relevant tools on the target device (these must be put under `$LISA_HOME/tools/<device_architecture/`)
- defining what trace events to collect

The `my_conf` dictionary declared below provides those information to the `TestEnv` initialization method.

In [ ]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"    : 'android',
    "board"       : 'hikey',

    # Folder where all the results will be collected
    "results_dir" : "Demo",

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
        "events" : [
            "cpu_frequency",
            "cpu_idle",
            "sched_switch",
            "sched_wakeup",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_overutilized",
        ],
        "buffsize" : 10 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'rt-app' ],
    
    # Define devlib modules to load
    "modules"     : [
        'cpufreq',       # enable CPUFreq support
        'cpuidle',       # enable cpuidle support
    ],
    
    # Comment this line to calibrate RTApp in your own platform
    "rtapp-calib" :  {"0": 251, "1": 251, "2": 250, "3": 251, "4": 253, "5": 251, "6": 251, "7": 252},
}

In [ ]:
# Support to access the remote target
import devlib
from env import TestEnv

!adb root
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

The `target` object allows to directly communicate with the target device. It is possible to query the device and execute commands on it, for example:

In [ ]:
# List device current directory
target.execute('ls')

or if you have specified the `devlib` modules to load during the Test Environment creation, you will now be able to read and information related to different subsystems.

For example, we have specified to load the `cpufreq` module. Therefore, we can read the current frequency of a CPU by running:

In [ ]:
# Read current frequency for CPU 0
target.cpufreq.get_frequency(0)

# Workload Execution

In this demo we generate a synthetic workload using `rt-app`. LISA provides an API to describe, create and run the synthetic workload. More specifically, there are two main classes of synthetic workloads available to the user, a Ramp task and a Periodic task.

Morever, as shown in the code below, it is possible to combine basic synthetic workloads into more complex ones.

In [ ]:
# Support for workload generation
from wlgen import RTA, Ramp, Periodic

# Initial phase and pinning parameters
ramp = Ramp(period_ms=100,
            start_pct=5,
            end_pct=65,
            delta_pct=20,
            time_s=1)

# Following phases
medium_slow = Periodic(duty_cycle_pct=10, duration_s=5, period_ms=100)
high_fast   = Periodic(duty_cycle_pct=60, duration_s=5, period_ms=10)

# Compose the task
complex_task = ramp + medium_slow + high_fast

The following method contains the code to:

1) Create an `rt-app` workload
2) Start and stop trace events collection using `ftrace`
3) Run the synthetic workload
4) Pull the trace(s) from the target device

In [ ]:
def experiment(te):

    # Create and RTApp RAMP task
    rtapp = RTA(te.target, 'ramp', calibration=te.calibration())
    rtapp.conf(kind='profile',
               params={
                    # Task 1
                    'complex' : complex_task.get(),
                    # Task 2
                    'ramp' : Ramp(period_ms=10,
                                  start_pct=60,
                                  end_pct=10,
                                  delta_pct=10,
                                  time_s=1).get(),
                    # Task 3
                    'periodic' : Periodic(duty_cycle_pct=40,
                                          duration_s=5,
                                          period_ms=200).get()
                },
                cpus=[1, 2])

    # FTrace the execution of this workload
    te.ftrace.start()
    rtapp.run(out_dir=te.res_dir)
    te.ftrace.stop()

    # Collect and keep track of the trace
    trace_file = os.path.join(te.res_dir, 'trace.dat')
    te.ftrace.get_trace(trace_file)
    
    # Collect and keep track of the Kernel Functions performance data
    stats_file = os.path.join(te.res_dir, 'trace.stats')
    te.ftrace.get_stats(stats_file)

    # Dump platform descriptor
    te.platform_dump(te.res_dir)

In [ ]:
experiment(te)

# Parse Trace and Profiling Data

Once the workload completes and trace(s) are pulled to the host, it is possible to perform statistical analysis on the trace events and generate a number of predefined plots.

LISA relies on `TRAPpy` (https://github.com/ARM-software/trappy) for trace parsing. What this tool does is to convert trace files data (generated by `FTrace` or `SysTrace`) into `pandas` dataframes (`pandas` is a Python package that provides a set of functionalities and data structures for data analysis).

In [ ]:
# Base folder where tests folder are located
res_dir = te.res_dir
logging.info('Content of the output folder %s', res_dir)
!tree {res_dir}

LISA provides a `Trace` objects to parse a set of events from a trace:

In [ ]:
with open(os.path.join(res_dir, 'platform.json'), 'r') as fh:
    platform = json.load(fh)

In [ ]:
# Support for trace analysis
from trace import Trace

trace_file = os.path.join(res_dir, 'trace.dat')
trace = Trace(platform, trace_file, events=my_conf['ftrace']['events'])

The `Trace` object contains two main attributes:

- an `analysis` object with several subobjects each referring to a different type of analysis (frequency analysis, idle states analysis, etc.)
- a `data_frame` object through which it is possible to access the data frame associated with each trace event as well as a set of predefined data frames which are basically the result of postprocessing the original trace events

Examples of data frame getters are:

In [ ]:
trace.data_frame.trace_event('cpu_frequency')

In [ ]:
trace.data_frame.cluster_frequency_residency('big')

# Trace visualization

The `TRAPpy` tool allows to visualize a trace in a similar way as `kernelshark` does:

In [ ]:
trappy.plotter.plot_trace(trace.ftrace)

It is possible to only plot a set of tasks by listing them as argument to the `plot_trace()` method:

In [ ]:
trappy.plotter.plot_trace(trace.ftrace, execnames=['ramp', 'periodic', 'complex'])

# Trace Analysis

## Tasks signals

One of the predefined plots in LISA allows you to visualize the behaviour over time of tasks related signals like PELT versus the CPU capacity.

Moreover, the pink bands shown in the plot below represent the intervals of time where the system was overutilized (i.e. out of EAS mode).

Last but not least, the residencies plot tells us on which CPU a particular task was running throughout the duration of the trace.

In [ ]:
trace.analysis.tasks.plotTasks('complex', signals=['util_avg', 'sched_overutilized', 'residencies'])

## Latency Plots

An interesting feature of LISA's trace analysis is the latency analysis module.

In particular, `plotLatency()` generates a set of plots to report the WAKEUP and PREEMPT latencies the specified task has been subject to. A WAKEUP latencies is the time from when a task becomes RUNNABLE till the first time it gets a CPU. A PREEMPT latencies is the time from when a RUNNABLE task is suspended because of the CPU is assigned to another task till when the task enters the CPU again.

In [ ]:
# Plot latency events for a specified task
latency_stats_df = trace.analysis.latency.plotLatency('ramp', )

In [ ]:
# Plot statistics on task latencies
latency_stats_df.T

The same plot is also available with a slighlty different representation. Here, the bands represent the latencies.

In [ ]:
# Plot latency events for a specified task
trace.analysis.latency.plotLatencyBands('ramp')

Tipically, the band representation of latencies is hard to analyse without narrowing the time range to see the bands clearly. To do this, we simply call the `setXTimeRange()` method available in the `trace` object to shrink the plot window to the specified time range.

In [ ]:
trace.setXTimeRange(t_min=3.9, t_max=4.1)
# Plot latency events for a specified task
trace.analysis.latency.plotLatencyBands('ramp')

In [ ]:
# Reset time range to full scale
trace.setXTimeRange(t_min=0, t_max=15)

## Frequency Analysis

The frequency analysis object does mainly analysis of `cpu_frequency` events. For example, the `plotClusterFrequencies()` method shows for each cluster how the frequency changes for the whole duration of the trace together with the average frequency.

In [ ]:
trace.analysis.frequency.plotClusterFrequencies()

Another interesting plot is the frequency residency, available per-cluster and per-CPU. Frequency residency is the amount of time spent by a given cluster/CPU at each available frequency.

In particular, these two methods show two types of residencies:

- total, the total amount of time spent by a cluster/CPU at a particular frequency
- active, the non-idle time spent by a cluster/CPU at a particular frequency

In [ ]:
trace.analysis.frequency.plotClusterFrequencyResidency()

Same information is also available in percentage with respect to the trace duration (or to the plot window in case the time range has been narrowed using `setXTimeRange()`.

In [ ]:
trace.analysis.frequency.plotClusterFrequencyResidency(pct=True)

## Idle Analysis

For what concerns idle states, the idle analysis module implements a set of methods to plot residencies in each idle state.

In [ ]:
trace.analysis.idle.plotClusterIdleStateResidency()

In [ ]:
trace.analysis.idle.plotClusterIdleStateResidency(pct=True)